Jason Cain, Nathan Harms, Marissa Puzan
## Serotonin transport modeling 



$$v_z \frac{\partial C}{\partial z} = D \frac{1}{r} \frac{\partial}{\partial r} \left( r \frac{\partial C}{\partial r} \right)$$


In [1]:
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from collections import namedtuple

In [2]:
%load_ext Cython

In [3]:
import numpy as np

from reactionKinetics import multistep
from scipy.integrate import odeint
from __future__ import division

from time import sleep


#Set up all the arguments for the ODE solver
def setup(length, radius, max_velocity, trypConditions, htpConditions, serConditions,
                kinetics, wallKinetics, rings, sections, timestep):

    serDiffusivity = serConditions[1] *3600 #6.2424e-8 m^2/sec
    serWallPerm = serConditions[2] *3600 #7.576e-13 m^2/sec
    trypDiffusivity = trypConditions[1]*3600 #5.386e-8 m^2/sec
    trypWallPerm = trypConditions[2]*3600 #6.44e-4 m^2/sec
    htpDiffusivity = htpConditions[1]*3600 #4.995e-8 m^2/sec
    htpWallPerm = htpConditions[2] *3600#estimating this as serotonin permeabiltiy - cannot find values for 5htp MLP


    permeabilities = [trypWallPerm, htpWallPerm, serWallPerm]
    diffusivities = [trypDiffusivity, htpDiffusivity, serDiffusivity]

    Concentrations = np.zeros((4, rings, sections))
    for i, J in enumerate([trypConditions, htpConditions, serConditions]):
        Concentrations[i, :, 0] = J[0]
    initialConcentrations = Concentrations.reshape(-1)
    
    argTuple = (diffusivities, permeabilities, length, radius, max_velocity*3600, kinetics, wallKinetics, rings, sections, timestep)

    return initialConcentrations, argTuple

def velocityProfile(max_velocity, radius, rings):
    r = np.linspace(0,radius,rings)
    velocityProfile = max_velocity*3600*(1-np.square(r)/np.square(radius))
    return velocityProfile



In [4]:
%%cython --annotate
import numpy as np
cimport numpy as np
from libc.math cimport log

import cython
@cython.cdivision(True)

cdef tuple multistep(tuple c, double vmax1, double Km1, double K1, double vmax2, double Km2, double K2):
    """
    c is a vector of concentrations of Tryptophan, 5-hydroxytryptophoan, and Serotonin
    based on Michaelis Menten kinetics
    Returns the rate dependent on the concentrations
    """
    cdef np.ndarray[np.float64_t, mode="c", ndim=2] cTry, c5HTP, cSer, dcTry, dc5HTP, dcSer
    cdef double k1fwd, k1rev, k2fwd, k2rev

    cTry, c5HTP, cSer = c
    k1fwd = vmax1/(Km1+cTry) #michaelus menton kinetics
    k1rev = k1fwd/K1 #equilibrium constant
    dcTry = -k1fwd*cTry + k1rev*c5HTP

    #Same reaction type for second reaction step
    k2fwd = vmax2/(Km2+c5HTP)
    k2rev = k2fwd/K2
    dcSer = k2fwd*c5HTP - k2rev*cSer

    dc5HTP = -dcTry - dcSer
    return dcTry, dc5HTP, dcSer

cdef tuple wall_multistep(tuple c, double vmax1, double Km1, double K1, double vmax2, double Km2, double K2):
    """
    c is a vector of concentrations of Tryptophan, 5-hydroxytryptophoan, and Serotonin
    based on Michaelis Menten kinetics
    Returns the rate dependent on the concentrations
    """

    cdef np.ndarray[np.float64_t, mode="c", ndim=1] cTry, c5HTP, cSer, dcTry, dc5HTP, dcSer
    cdef double k1fwd, k1rev, k2fwd, k2rev


    cTry, c5HTP, cSer = c
    k1fwd = vmax1/(Km1+cTry) #michaelus menton kinetics
    k1rev = k1fwd/K1 #equilibrium constant
    dcTry = -k1fwd*cTry + k1rev*c5HTP

    #Same reaction type for second reaction step
    k2fwd = vmax2/(Km2+c5HTP)
    k2rev = k2fwd/K2
    dcSer = k2fwd*c5HTP - k2rev*cSer

    dc5HTP = -dcTry - dcSer
    return dcTry, dc5HTP, dcSer


cpdef np.ndarray[np.float64_t, mode="c", ndim=1] getConcentration(
                     np.ndarray[np.float64_t, ndim=1] concentrationVector,
                     float time,
                     list diffusivities, 
                     list permeabilities,
                     double length, 
                     double radius, 
                     double max_velocity,
                     tuple kinetics, 
                     tuple wallKinetics,
                     int rings, 
                     int sections,
                     double timestep):
    
    cdef tuple concentrationTuple
    cdef np.ndarray[np.float64_t, mode="c", ndim=3] concentrations, rates, lapZ, lapR, deltaZ, deltaR
    cdef np.ndarray[np.float64_t, mode="c", ndim=2] trypConcentration, htpConcentration, serConcentration
    cdef np.ndarray[np.float64_t, mode="c", ndim=2] tryp_rate, htp_rate, ser_rate, uptakeRate
    cdef np.ndarray[np.float64_t, mode="c", ndim=1] velocityProf, r
    cdef double outerRingSA, dz, dr
    cdef double vmax1, Km1, K1, vmax2, Km2, K2, wall_vmax1, wall_Km1, wall_K1, wall_vmax2, wall_Km2, wall_K2
    
    vmax1, Km1, K1, vmax2, Km2, K2 = kinetics
    wall_vmax1, wall_Km1, wall_K1, wall_vmax2, wall_Km2, wall_K2 = wallKinetics

    outerRingSA = np.pi*2*radius*length/sections
    
    r = np.linspace(0,radius,rings)
    velocityProf = max_velocity*3600*(1-np.square(r)/np.square(radius))
    
    dz = length/(sections-1)
    dr = radius/(rings-1)

    concentrations = concentrationVector.reshape((4,rings,sections))
    trypConcentration = concentrations[0]
    htpConcentration = concentrations[1]
    serConcentration = concentrations[2]


    #This is so I can track the rate of uptake and let odeint calculate the flux
    uptakeRate = np.zeros_like(concentrations[3])

    concentrationList = [trypConcentration, htpConcentration, serConcentration]
    # print(concentrationsVector)

    lapZ = np.diff(concentrations, n = 2, axis = 1)
    lapR = np.diff(concentrations, n = 2, axis = 2)

    deltaZ = np.diff(concentrations, n = 1, axis = 1)
    deltaR = np.diff(concentrations, n = 1, axis = 2)

    concentrationTuple = (trypConcentration, htpConcentration, serConcentration)

    #Rates from reaction kinetics
    tryp_rate, htp_rate, ser_rate = multistep(concentrationTuple, vmax1, Km1, K1, vmax2, Km2, K2)
    wallConcentrationTuple = (trypConcentration[ -1 , : ], htpConcentration[ -1 , : ], serConcentration[ -1 , : ])
    try_wallRate, htp_wallRate, ser_wallRate = wall_multistep(wallConcentrationTuple, wall_vmax1, wall_Km1, wall_K1, wall_vmax2, wall_Km2, wall_K2)
    tryp_rate[ -1 , : ], htp_rate[ -1 , : ], ser_rate[ -1 , : ] = try_wallRate, htp_wallRate, ser_wallRate

    for i, rate in enumerate([tryp_rate, htp_rate, ser_rate]): #Zip would be more confusing here I think....

        # Convection!
        convection = np.zeros((rings, sections))
        for j in range(rings-1):
            convection[j] += velocityProf[j] * deltaZ[i,j] #addition to make sure sizes are correct
        rate -= convection

        # Diffusion!
        rate[1:-1,:] += diffusivities[i]*lapZ[i]/(dz*dz)
        rate[:,1:-1] += diffusivities[i]*lapR[i]/(dr*dr)

        # Boundaries!
        rate[-1,:] += permeabilities[i]*wallConcentrationTuple[i]/radius
        if i == 2:
            #i dont know how to maket his better...
            uptakeRate[-1,:] += 2*np.pi*dz*radius*rate[-1,:]
        rate[:,-1] += 2 * diffusivities[i] * (concentrations[i, :,-2] - concentrations[i, :,-1] ) / (dz*dz)
        rate[0,:] += 2 * diffusivities[i] * (concentrations[i, 1, :] - concentrations[i, 0, :] ) / (dr*dr)
        rate[:,0] += 2 * diffusivities[i] * (concentrations[i, :, 1] - concentrations[i, :, 0] ) / (dz*dz)

    rates = np.stack((tryp_rate, htp_rate, ser_rate, uptakeRate))

    return rates.reshape(-1)

In [5]:
from scipy.integrate import odeint

def solveODE(concentrationVector, times, argTuple):
    results = odeint(getConcentration, concentrationVector, times, args=argTuple)
    
    return results.reshape(-1,4,rings,sections)

In [6]:

KineticsParamers = namedtuple('ConditionSet', ['vmax1', 'Km1', 'K1', 'vmax2', 'Km2', 'K2'])


serCondition = (2, 6.2424e-8, 7.576e-13) #Concentration, Diffusivity, Permeability

trypCondition = (.1,5.386e-8,6.44e-4)

htpCondition = (0,4.995e-8,7.576e-13)

kinetics = (0.1868167, 1.43533284,0.43680929, 9.97704964, 2.37430847, 0.25340153)# vmax1, Km1, K1, vmax2, Km2, K2

wallKinetics = (7.56e-14,0.6e-3,1e5,7.14,119,1e5) # 1e5 -> irreversible assumption

radius = 2.5/2/100
length = 7.5
max_velocity = .0287/60
timestep = 20/3600
rings = 20
sections = 500


initialConcentration, argTuple = setup(length, radius, max_velocity, 
                 trypCondition, htpCondition, serCondition, 
                 kinetics, wallKinetics, rings, sections, timestep)

timeVector = np.arange(0, length/max_velocity/3600, timestep)
results = solveODE(initialConcentration, timeVector, argTuple)

TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
results = solveODE(argTuple)

In [ ]:
results = Model.results.reshape(-1,3,rings,sections)
print(results.shape)
serotonin = results[:,2,:,:]
tryptophan = results[:,0,:,:]
htp = results[:,1,:,:]
serotoninUptake = results[:,3,:,:]


In [ ]:
plt.plot(serotoninUptake)

In [ ]:
from matplotlib import animation
import matplotlib
import seaborn as sns


matplotlib.rc('animation', html='html5')
fig = plt.figure()
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
timestamp = fig.text(0.45,0.1,'timestamp')
frames = 100
def animate(i):
    step = i * len(serotonin[:,0,0])//(frames)
    serotonin_g = serotonin[i,:,:50]
    tryp_g = tryptophan[i,:,:50]
    sns.heatmap(serotonin_g, vmin=0,vmax=2, ax=ax1, cbar = None)
    ax1.set_title('Serotonin')
    sns.heatmap(tryp_g, vmin=0,vmax=2, ax=ax2, cbar=None)
    ax2.set_title('Tryptophan')
    timestamp.set_text('t={0:.0f} hours'.format(Model.times[i]))
    plt.tight_layout()
    
anim = animation.FuncAnimation(fig, animate, frames=frames, repeat_delay=2000, repeat=True)
anim

In [ ]:
from SALib.sample.saltelli import sample as ss
from SALib.analyze.sobol import analyze as sa


In [ ]:
def laminarModel(data):
    trypCondition = ConditionSet(
                Concentration = data[3],
                Diffusivity = data[4],
                Permeability = data[5])

    htpCondition = ConditionSet(
                Concentration = data[6],
                Diffusivity = data[7],
                Permeability = data[8])

    serCondition = ConditionSet(
                Concentration = data[9],
                Diffusivity = data[10],
                Permeability = data[11])
    
    kinetics = KineticsParamers(vmax1 = data[12],
                            Km1 = data[13],
                            K1 = data[14],
                            vmax2 = data[15],
                            Km2 = data[16],
                            K2 = data[17])
    
    wallKinetics = KineticsParamers(vmax1 = data[18],
                            Km1 = data[19],
                            K1 = data[20],
                            vmax2 = data[21],
                            Km2 = data[22],
                            K2 = data[23])
    
    rings = 20
    sections = 500
    
    Model = laminarFlow.LaminarFlow(data[0],data[1],data[2],
                                    trypCondition, htpCondition, serCondition, 
                                    kinetics, wallKinetics, 
                                    rings, sections)

In [ ]:


# morris_problem = {
#     # There are six variables
#     'num_vars': 23,
#     # These are their names
#     'names': ['length', 'radius', 'max_velocity', 'serConc', 'serDiff', 
#               'serPerm', 'tryConc', 'tryDiff', 'tryPerm','vmax1', 'Km1', 'K1', 'vmax2', 'Km2', 'K2'],
#     # These are their plausible ranges over which we'll move the variables
#     'bounds': [[,], # length (m)
#                [,], # radius (m)
#                [,], # max_velocity (m/s)
#                [,], # Tryptophan Concentration
#                [,], # Tryptophan Diffusivity
#                [,], # Tryptophan Wall Permabilityhttp://localhost:8888/notebooks/SerotoninTransportModel.ipynb#
#                [,], # 5HTP Concentration
#                [,], # 5HTP Diffusivity
#                [,], # 5HTP Wall Permability
#                [,], # Serotonin Concentration (mM)
#                [,], # Serotonin Diffusivity 
#                [,], # Serotonin Wall Permeability 
#                [,], # Max rate of Tryp -> 5HTP
#                [,], # Michaelus Menton Constant Tryp -> 5HTP
#                [,], # Equilibrium Constant for 5HTP and Tryp
#                [,], # Max rate of 5HTP -> Serotonin
#                [,], # Michaelus Menton Constant 5HTP -> Serotonin
#                [,], # Equilibrium Constant for 5HTP and Serotonin
#                [,], # Max wall rate of Tryp -> 5HTP
#                [,], # Michaelus Menton Constant @ wall Tryp -> 5HTP
#                [,], # Equilibrium Constant for 5HTP and Tryp @ wall
#                [,], # Max rate of 5HTP -> Serotonin @ wall
#                [,], # Michaelus Menton Constant 5HTP -> Serotonin @ wall
#                [,], # Equilibrium Constant for 5HTP and Serotonin @ wall

#               ],
#     # I don't want to group any of these variables together
#     'groups': None
#     }